# DSCI: 525 Milestone 1 - Group 8

## Rachel Wong, Rui Wang, Daniel Ortiz, Santiago Rugeles Schoonewolff

### Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

# Dask
import dask.dataframe as dd

# pyarrow and feather
import pyarrow.feather as feather
import pyarrow.dataset as ds

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

### Downloading the data

In [3]:
# Necessary metadata
article_id = 14096681  # unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

### Unzipping the data

In [5]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 2.47 s, sys: 2.26 s, total: 4.73 s
Wall time: 1min 18s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 18 s, sys: 2.45 s, total: 20.5 s
Wall time: 20.9 s


### Combining data CSVs using Pandas

In [7]:
df = pd.read_csv("./figshareairline/ACCESS-CM2_daily_rainfall_NSW.csv")
df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


In [8]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together
## here we are using a normal python way of merging the data 

files = glob.glob('figshareairline/*.csv') # load all the CSVs
df = pd.concat((pd.read_csv(file, index_col=0) # combine them all
                .assign(model=re.findall(r'/([^_]*)', file)[0])
                for file in files)
              )
df.to_csv("figshareairline/combined_data.csv")

peak memory: 404.33 MiB, increment: 0.05 MiB
CPU times: user 21min 42s, sys: 2min 20s, total: 24min 3s
Wall time: 25min 31s


In [9]:
df_combined = pd.read_csv("./figshareairline/combined_data.csv")
df_combined # combined dataframe 

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...,...
251922039,2014-12-27 12:00:00,-30.157068,-29.214660,153.1250,154.3750,6.689683e+00,SAM0-UNICON
251922040,2014-12-28 12:00:00,-30.157068,-29.214660,153.1250,154.3750,7.862555e+00,SAM0-UNICON
251922041,2014-12-29 12:00:00,-30.157068,-29.214660,153.1250,154.3750,1.000503e+01,SAM0-UNICON
251922042,2014-12-30 12:00:00,-30.157068,-29.214660,153.1250,154.3750,8.541592e+00,SAM0-UNICON


discussion about the different computer times on each of our machines

In [10]:
df_combined["model"].unique() # print out the unique models

array(['MPI-ESM-1-2-HAM', 'AWI-ESM-1-1-LR', 'NorESM2-LM', 'ACCESS-CM2',
       'FGOALS-f3-L', 'CMCC-CM2-HR4', 'MRI-ESM2-0', 'GFDL-CM4',
       'BCC-CSM2-MR', 'EC-Earth3-Veg-LR', 'CMCC-ESM2', 'NESM3',
       'MPI-ESM1-2-LR', 'ACCESS-ESM1-5', 'FGOALS-g3', 'INM-CM4-8',
       'MPI-ESM1-2-HR', 'TaiESM1', 'NorESM2-MM', 'CMCC-CM2-SR5',
       'combined', 'observed', 'KIOST-ESM', 'INM-CM5-0', 'MIROC6',
       'BCC-ESM1', 'GFDL-ESM4', 'CanESM5', 'SAM0-UNICON'], dtype=object)

In [11]:
%%sh
du -sh figshareairline/combined_data.csv

 22G	figshareairline/combined_data.csv


We can see from our combined dataframe that we have 29 unique models to continue our analysis with.

### Load the combined CSV to memory and perform a simple EDA

In [ ]:
%%time
%%memit
#simple pandas - This is how we do normally ,which means we are loading the entire data to the memory
df = pd.read_csv("figshareairline/combined_data.csv")
print(df["model"].value_counts())

In [ ]:
df.head()

In [ ]:
# checking datatypes for columns
df.dtypes

We can see that we have object and float64 type columns in our dataframe. This makes sense that `time` and `model` are object types and the rest such as latitude, longitude, and rain are float64 types.

### Investigate changing the `dtype` of our data

In [ ]:
print(f"The memory usage with the original float64 dtype: {df[['lat_min','lat_max','rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"The memory usage after changing to float32 dtype: {df[['lat_min','lat_max','rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

discussion about reducing memory usage of our numeric data types by switching from float64 to float32

### Loading our data using Dask and checking the value counts of models

In [ ]:
%%time
%%memit
# Dask
df_dask = dd.read_csv('figshareairline/combined_data.csv')
print(df_dask["model"].value_counts().compute())

discussion about reducing memory usage with Dask

### Transfering the dataframe from Python to R using Feather

In [ ]:
%%time
%%memit
dataset = ds.dataset("figshareairline/combined_data.csv", format="csv")
## this is of arrow table format
table = dataset.to_table()

In [ ]:
%%time
# writing in feather format
feather.write_feather(table, 'figshareairline/combined_data.feather')

discussion of memory and time using feather, and why we chose feather

### Simple EDA in R

In [ ]:
%%R
library(tidyr)

In [ ]:
%%time
%%R
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figshareairline/combined_data.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model) # showing the different counts of the models 
end_time <- Sys.time()
print(result)
print(end_time - start_time)

In [ ]:
%%R
result <- r_table %>% count(time) # showing the different counts of the time
print(result)

discussion about the different counts using R, time and memory differences

In [ ]:
%%R
r_table_d <- r_table %>% drop_na() # drop NA values

In [ ]:
%%R
r_table_d <- r_table_d %>% rename(rain_mmperday = `rain (mm/day)`) # rename the column for rain

In [ ]:
%%R
Columns <- c("lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/perday)")
Mean <- c(mean(r_table_d$lat_min), mean(r_table_d$lat_max), mean(r_table_d$lon_min), mean(r_table_d$lon_max), mean(r_table_d$rain_mmperday))
Mode <- c(mode(r_table_d$lat_min), mode(r_table_d$lat_max), mode(r_table_d$lon_min), mode(r_table_d$lon_max), mode(r_table_d$rain_mmperday))
Median <- c(median(r_table_d$lat_min), median(r_table_d$lat_max), median(r_table_d$lon_min), median(r_table_d$lon_max), median(r_table_d$rain_mmperday))

result <- data.frame(Columns, Mean, Mode, Median)
print(result)

discussion about the different mean, median, modes of the columns analyzed using R